Cell 1 — Imports & Config

In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from typing import Dict, Tuple, List
import hashlib

# Paths
DATASET_ROOT = Path(r"E:\0.TA_Teguh\GMM Trial 2")
HEAD1_DIR = DATASET_ROOT / "Head 1"
OUT_ROOT = Path(r"E:\0.TA_Teguh\Sampling\Weighted")

# Subjects A..J
SUBJECTS = list("ABCDEFGHIJ")

# Candidates
N_TARGETS = [48, 56, 64]
RUN_N_TARGETS = [64]  # start with one; change to [48,56,64] after sanity check

# Must-match Head-1 columns
COLS_EXPECTED = ["timestamp","frame","x","y","z","doppler","SNR","w_snr","d2","is_inlier"]
COLS_OUT = COLS_EXPECTED[:]  # keep order

# Sampling settings
EPS_W = 1e-12

# Read/write config
CSV_READ_KWARGS = dict(dtype={
    "timestamp": str,
    "frame": np.int32,
    "x": np.float32,
    "y": np.float32,
    "z": np.float32,
    "doppler": np.float32,
    "SNR": np.float32,
    "w_snr": np.float32,
    "d2": np.float32,
    "is_inlier": np.int8,
})
CSV_WRITE_KWARGS = dict(index=False)

print("OK | HEAD1_DIR:", HEAD1_DIR)
print("OK | OUT_ROOT :", OUT_ROOT)
print("RUN_N_TARGETS :", RUN_N_TARGETS)


OK | HEAD1_DIR: E:\0.TA_Teguh\GMM Trial 2\Head 1
OK | OUT_ROOT : E:\0.TA_Teguh\Sampling\Weighted
RUN_N_TARGETS : [64]


Cell 2 — Deterministic seed helper

In [2]:
def stable_seed(subject: str, file_name: str, frame: int, n_target: int) -> int:
    s = f"{subject}|{file_name}|{frame}|{n_target}"
    h = hashlib.sha256(s.encode("utf-8")).hexdigest()
    return int(h[:8], 16)  # 32-bit


Cell 3 — IO helpers (list files, read head1)

In [3]:
def list_jalan_csvs(subject_dir: Path) -> List[Path]:
    if not subject_dir.exists():
        return []
    return sorted(subject_dir.glob("Jalan*.csv"))

def read_head1_csv(path: Path) -> pd.DataFrame:
    df = pd.read_csv(path, **CSV_READ_KWARGS)
    missing = [c for c in COLS_EXPECTED if c not in df.columns]
    if missing:
        raise ValueError(f"Missing columns in {path.name}: {missing}")
    df = df[COLS_EXPECTED].copy()
    return df


Cell 4 — Frame extraction & sanitization

In [4]:
def get_frame_df(df: pd.DataFrame, frame_id: int) -> pd.DataFrame:
    f = df[df["frame"] == frame_id].copy()
    if f.empty:
        return f
    num_cols = ["x","y","z","doppler","SNR","w_snr","d2"]
    for c in num_cols:
        v = f[c].to_numpy()
        ok = np.isfinite(v)
        if not ok.all():
            f = f.loc[ok].copy()
    return f


Cell 5 — Core weighted index sampler (with fallback)

In [5]:
def _normalize_probs(w: np.ndarray) -> Tuple[np.ndarray, bool]:
    w = np.asarray(w, dtype=np.float64)
    w = np.clip(w, 0.0, None)
    s = w.sum()
    if (not np.isfinite(s)) or s <= EPS_W:
        return np.array([], dtype=np.float64), False
    return (w / s), True

def weighted_choice_indices(w: np.ndarray, k: int, replace: bool, seed: int) -> np.ndarray:
    """
    Weighted sampling indices.
    - replace=False for downsample
    - replace=True for upsample
    Falls back to uniform if weights degenerate.
    """
    if k <= 0:
        return np.empty((0,), dtype=np.int32)

    n = len(w)
    rng = np.random.default_rng(seed)

    p, ok = _normalize_probs(w)
    if not ok:
        # uniform fallback
        return rng.choice(n, size=k, replace=replace).astype(np.int32)

    return rng.choice(n, size=k, replace=replace, p=p).astype(np.int32)


Cell 6 — Per-frame weighted sampler (downsample/upsample/equal)

In [6]:
def sample_frame_weighted(frame_df: pd.DataFrame, n_target: int, subject: str, file_name: str) -> Tuple[pd.DataFrame, Dict]:
    M = len(frame_df)
    meta = {"M": M, "N": n_target, "action": None, "n_drop": 0, "n_fill": 0, "fallback_uniform": 0}

    if M == 0:
        meta["action"] = "empty"
        return frame_df[COLS_OUT].copy(), meta

    fr = int(frame_df["frame"].iloc[0])
    seed = stable_seed(subject, file_name, fr, n_target)

    w = frame_df["w_snr"].to_numpy()

    if M == n_target:
        meta["action"] = "equal"
        return frame_df[COLS_OUT].copy(), meta

    if M > n_target:
        idx = weighted_choice_indices(w, k=n_target, replace=False, seed=seed)
        out = frame_df.iloc[idx].copy()
        meta["action"] = "downsample"
        meta["n_drop"] = M - n_target
        # detect fallback roughly: if weights degenerate, our helper uses uniform
        p, ok = _normalize_probs(w)
        meta["fallback_uniform"] = 0 if ok else 1
        return out[COLS_OUT], meta

    # M < n_target (upsample): keep all + add (N-M) with replacement
    k = n_target - M
    add_idx = weighted_choice_indices(w, k=k, replace=True, seed=seed)
    add = frame_df.iloc[add_idx].copy()
    out = pd.concat([frame_df, add], axis=0, ignore_index=True)
    meta["action"] = "upsample"
    meta["n_fill"] = k
    p, ok = _normalize_probs(w)
    meta["fallback_uniform"] = 0 if ok else 1
    return out[COLS_OUT], meta


Cell 7 — Process one file (loop all frames, save CSV)

In [7]:
def process_one_file(head1_path: Path, subject: str, n_target: int) -> Dict:
    df = read_head1_csv(head1_path)

    file_name = head1_path.name
    out_dir = OUT_ROOT / f"N{n_target}" / subject
    out_dir.mkdir(parents=True, exist_ok=True)
    out_path = out_dir / file_name

    frames = df["frame"].unique()
    frames.sort()

    out_chunks = []
    stats = {
        "file": file_name, "subject": subject, "N": n_target,
        "frames": int(len(frames)),
        "down": 0, "up": 0, "eq": 0, "empty": 0,
        "total_drop": 0, "total_fill": 0,
        "fallback_uniform_frames": 0
    }

    for fr in frames:
        fdf = get_frame_df(df, int(fr))
        sampled, meta = sample_frame_weighted(fdf, n_target=n_target, subject=subject, file_name=file_name)

        out_chunks.append(sampled)

        act = meta["action"]
        if act == "downsample":
            stats["down"] += 1
            stats["total_drop"] += int(meta["n_drop"])
        elif act == "upsample":
            stats["up"] += 1
            stats["total_fill"] += int(meta["n_fill"])
        elif act == "equal":
            stats["eq"] += 1
        else:
            stats["empty"] += 1

        if meta.get("fallback_uniform", 0) == 1:
            stats["fallback_uniform_frames"] += 1

    out_df = pd.concat(out_chunks, axis=0, ignore_index=True)
    out_df.to_csv(out_path, **CSV_WRITE_KWARGS)
    return stats


Cell 8 — Run all (batch: subject → file) + summary CSV

In [8]:
all_stats = []

for n_target in RUN_N_TARGETS:
    for subject in SUBJECTS:
        subj_dir = HEAD1_DIR / subject
        files = list_jalan_csvs(subj_dir)
        if not files:
            print(f"[WARN] No files for subject {subject} in {subj_dir}")
            continue

        print(f"\n=== N={n_target} | Subject {subject} | files={len(files)} ===")
        for p in files:
            st = process_one_file(p, subject=subject, n_target=n_target)
            all_stats.append(st)
            print(f"  {st['file']}: frames={st['frames']}, down={st['down']}, up={st['up']}, eq={st['eq']}, "
                  f"fill={st['total_fill']}, drop={st['total_drop']}, fallback_uniform={st['fallback_uniform_frames']}")

summary_df = pd.DataFrame(all_stats)
summary_path = OUT_ROOT / "weighted_sampling_summary.csv"
summary_df.to_csv(summary_path, index=False)
print("\nSaved summary:", summary_path)

summary_df.head()



=== N=64 | Subject A | files=72 ===
  Jalan1.csv: frames=82, down=7, up=74, eq=1, fill=2662, drop=100, fallback_uniform=0
  Jalan10.csv: frames=87, down=20, up=66, eq=1, fill=2257, drop=375, fallback_uniform=0
  Jalan11.csv: frames=87, down=33, up=54, eq=0, fill=2002, drop=1044, fallback_uniform=0
  Jalan12.csv: frames=98, down=34, up=62, eq=2, fill=2085, drop=615, fallback_uniform=0
  Jalan13.csv: frames=103, down=28, up=74, eq=1, fill=2137, drop=568, fallback_uniform=0
  Jalan14.csv: frames=81, down=3, up=78, eq=0, fill=2965, drop=31, fallback_uniform=0
  Jalan15.csv: frames=239, down=40, up=196, eq=3, fill=6940, drop=603, fallback_uniform=0
  Jalan16.csv: frames=229, down=45, up=181, eq=3, fill=6298, drop=779, fallback_uniform=0
  Jalan17.csv: frames=194, down=67, up=126, eq=1, fill=3688, drop=1635, fallback_uniform=0
  Jalan18.csv: frames=166, down=91, up=71, eq=4, fill=1670, drop=3188, fallback_uniform=0
  Jalan19.csv: frames=222, down=53, up=169, eq=0, fill=5668, drop=1310, fall

,file,subject,N,frames,down,up,eq,empty,total_drop,total_fill,fallback_uniform_frames
0,Jalan1.csv,A,64,82,7,74,1,0,100,2662,0
1,Jalan10.csv,A,64,87,20,66,1,0,375,2257,0
2,Jalan11.csv,A,64,87,33,54,0,0,1044,2002,0
3,Jalan12.csv,A,64,98,34,62,2,0,615,2085,0
4,Jalan13.csv,A,64,103,28,74,1,0,568,2137,0


Cell 9 — Quick aggregate check (optional)

In [9]:
if len(all_stats) == 0:
    print("No stats collected. Run the batch cell first.")
else:
    df_sum = pd.DataFrame(all_stats)

    # 1) Aggregate per N (overall)
    agg_N = (
        df_sum.groupby(["N"], as_index=False)
        .agg(
            files=("file", "count"),
            frames=("frames", "sum"),
            down=("down", "sum"),
            up=("up", "sum"),
            eq=("eq", "sum"),
            empty=("empty", "sum"),
            total_fill=("total_fill", "sum"),
            total_drop=("total_drop", "sum"),
            fallback_uniform_frames=("fallback_uniform_frames", "sum"),
        )
        .sort_values(["N"])
    )

    # 2) Aggregate per N + subject
    agg_NS = (
        df_sum.groupby(["N", "subject"], as_index=False)
        .agg(
            files=("file", "count"),
            frames=("frames", "sum"),
            down=("down", "sum"),
            up=("up", "sum"),
            eq=("eq", "sum"),
            empty=("empty", "sum"),
            total_fill=("total_fill", "sum"),
            total_drop=("total_drop", "sum"),
            fallback_uniform_frames=("fallback_uniform_frames", "sum"),
        )
        .sort_values(["N", "subject"])
    )

    display(agg_N)
    display(agg_NS)


,N,files,frames,down,up,eq,empty,total_fill,total_drop,fallback_uniform_frames
0,64,720,114170,28604,84497,1069,0,2842765,652901,0


,N,subject,files,frames,down,up,eq,empty,total_fill,total_drop,fallback_uniform_frames
0,64,A,72,10459,2787,7574,98,0,257165,65306,0
1,64,B,72,12224,3030,9080,114,0,317323,70511,0
2,64,C,72,13890,2692,11087,111,0,382133,54379,0
3,64,D,72,11861,2514,9235,112,0,316163,56491,0
4,64,E,72,11102,2931,8074,97,0,267116,66625,0
5,64,F,72,10424,2682,7639,103,0,256649,60975,0
6,64,G,72,10532,2766,7685,81,0,254437,64158,0
7,64,H,72,11352,2599,8646,107,0,303019,58607,0
8,64,I,72,10993,3589,7277,127,0,227240,88850,0
9,64,J,72,11333,3014,8200,119,0,261520,66999,0
